In [26]:
%runfile code.py

# Computing the stable bimodule character of diagonal harmonic polynomials

The $GL_n$ character for a given irreducible representation of $S_n$
(fetched from database if not already calculated):

In [13]:
harmonic_character([3,2])

s[2] + s[2, 1] + s[2, 2] + s[3] + s[3, 1] + s[4] + s[4, 1] + s[5] + s[6]

Trigger the calculation of the above for all representations of $S_n$:

In [14]:
harmonic_characters(3)

[3]	(0:00:00): s[]
[2, 1]	(0:00:00): s[1] + s[2]
[1, 1, 1]	(0:00:00): s[1, 1] + s[3]


Compute all the currently computed terms of the $GL_r\times S_n$ character of diagonal harmonic polynomials:

In [15]:
harmonic_bicharacter_truncated_series()

s[] # s[] + s[] # s[1] + s[] # s[2] + s[] # s[3] + s[] # s[4] + s[] # s[5] + s[] # s[6] + s[1] # s[1, 1] + s[1] # s[2, 1] + s[1] # s[3, 1] + s[1] # s[4, 1] + s[1] # s[5, 1] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1, 1] + s[1, 1] # s[3, 1, 1] + s[1, 1] # s[4, 1, 1] + s[1, 1, 1] # s[1, 1, 1, 1] + s[1, 1, 1] # s[2, 1, 1, 1] + s[1, 1, 1] # s[3, 1, 1, 1] + s[1, 1, 1, 1] # s[1, 1, 1, 1, 1] + s[1, 1, 1, 1] # s[2, 1, 1, 1, 1] + s[1, 1, 1, 1, 1] # s[1, 1, 1, 1, 1, 1] + s[2] # s[2, 1] + s[2] # s[2, 2] + s[2] # s[3, 1] + s[2] # s[3, 2] + s[2] # s[4, 1] + s[2] # s[4, 2] + s[2] # s[5, 1] + s[2, 1] # s[2, 1, 1] + s[2, 1] # s[2, 2] + s[2, 1] # s[2, 2, 1] + s[2, 1] # s[3, 1, 1] + s[2, 1] # s[3, 2] + s[2, 1] # s[3, 2, 1] + s[2, 1] # s[4, 1, 1] + s[2, 1] # s[4, 2] + s[2, 1, 1] # s[2, 1, 1, 1] + s[2, 1, 1] # s[2, 2, 1] + s[2, 1, 1] # s[2, 2, 1, 1] + s[2, 1, 1] # s[3, 1, 1, 1] + s[2, 1, 1] # s[3, 2, 1] + s[2, 1, 1, 1] # s[2, 1, 1, 1, 1] + s[2, 1, 1, 1] # s[2, 2, 1, 1] + s[2, 2] # s[2, 2, 1] + s[2, 2] # s[

# Computing the space of harmonics manually

In [16]:
n = 5
r = 2
P = DiagonalPolynomialRing(QQ,n,r)
X = P.algebra_generators(); X

[x00 x01 x02 x03 x04]
[x10 x11 x12 x13 x14]

In [17]:
Degrees = cartesian_product([ZZ]*r)
def add_degree((d1,mu), d2):
    d = d1+d2
    if not all(i>=0 for i in d):
        raise ValueError("Invalid degree")
    return d,mu

R = QQ['q,t']
Sym = SymmetricFunctions(R.fraction_field())
Sym.inject_shorthands()
def hilbert_parent(dimensions):
    return s.sum_of_terms( (mu, R({d:dim})) for (d,mu),dim in dimensions.iteritems() )

/opt/sage-git2/local/lib/python2.7/site-packages/sage/combinat/sf/sf.py:1416: RuntimeWarning: redefining global value `e`
  inject_variable(shorthand, getattr(self, shorthand)())


In [18]:
R({Degrees([2,1]):3})

3*q^2*t

In [19]:
dimensions = {(Degrees([2,1]), Partition([3,2])): 42,
              (Degrees([0,0]), Partition([2])): 5}
hilbert_parent(dimensions)

5*s[2] + 42*q^2*t*s[3, 2]

In [20]:
generators = {}
for mu in Partitions(n):
    for t in StandardTableaux(mu):
        p = P.higher_specht(t, harmonic=True)
        d = (Degrees([p.degree()]+[0]*(r-1)), mu)
        if d not in generators:
            generators[d] = []
        generators[d].append(p)
for key,value in generators.iteritems():
    print key, [factor(p) for p in value]

((5, 0), [3, 2]) [(2/25) * (x01 - x04) * (-x00 + x03) * (x00^3 - 2*x00^2*x01 - 2*x00*x01^2 + x01^3 - 2*x00^2*x02 + 6*x00*x01*x02 - 2*x01^2*x02 + 3*x00*x02^2 + 3*x01*x02^2 - 4*x02^3 + 3*x00^2*x03 + x00*x01*x03 - 2*x01^2*x03 - 14*x00*x02*x03 + 6*x01*x02*x03 + 3*x02^2*x03 + 3*x00*x03^2 - 2*x01*x03^2 - 2*x02*x03^2 + x03^3 - 2*x00^2*x04 + x00*x01*x04 + 3*x01^2*x04 + 6*x00*x02*x04 - 14*x01*x02*x04 + 3*x02^2*x04 + x00*x03*x04 + x01*x03*x04 + 6*x02*x03*x04 - 2*x03^2*x04 - 2*x00*x04^2 + 3*x01*x04^2 - 2*x02*x04^2 - 2*x03*x04^2 + x04^3)]
((3, 0), [3, 2]) [(4/5) * (x01 - x04) * (-x00 + x03) * (x00 + x01 - 4*x02 + x03 + x04)]
((4, 0), [3, 1, 1]) [(-2/5) * (x03 - x04) * (-x00 + x03) * (x00 - x04) * (2*x00 - 3*x01 - 3*x02 + 2*x03 + 2*x04)]
((6, 0), [3, 1, 1]) [(1/20) * (x03 - x04) * (-x00 + x03) * (x00 - x04) * (x00^2*x01 - 2*x01^3 + x00^2*x02 - 8*x00*x01*x02 + 6*x01^2*x02 + 6*x01*x02^2 - 2*x02^3 - x00^2*x03 + 3*x00*x01*x03 + 3*x00*x02*x03 - 8*x01*x02*x03 - x00*x03^2 + x01*x03^2 + x02*x03^2 - x00^2*x

In [21]:
operators = P.polarization_operators_by_degree(side="down"); operators

{(-3, 1): [<functools.partial object at 0x7f1bc22d2628>],
 (-1, 1): [<functools.partial object at 0x7f1bc22e5050>],
 (-4, 1): [<functools.partial object at 0x7f1bc22d27e0>],
 (-2, 1): [<functools.partial object at 0x7f1bc22d2788>]}

In [22]:
F = Subspace(generators, operators=operators, add_degrees=add_degree)
F._hilbert_parent = hilbert_parent

In [23]:
%time dims = F.dimensions()

CPU times: user 2.37 s, sys: 24 ms, total: 2.4 s
Wall time: 2.38 s


In [24]:
F.hilbert_polynomial()

(q^10+q^9*t+q^8*t^2+q^7*t^3+q^6*t^4+q^5*t^5+q^4*t^6+q^3*t^7+q^2*t^8+q*t^9+t^10+q^8*t+q^7*t^2+q^6*t^3+q^5*t^4+q^4*t^5+q^3*t^6+q^2*t^7+q*t^8+q^7*t+2*q^6*t^2+2*q^5*t^3+2*q^4*t^4+2*q^3*t^5+2*q^2*t^6+q*t^7+q^6*t+q^5*t^2+2*q^4*t^3+2*q^3*t^4+q^2*t^5+q*t^6+q^4*t^2+q^3*t^3+q^2*t^4)*s[1, 1, 1, 1, 1] + (q^9+q^8*t+q^7*t^2+q^6*t^3+q^5*t^4+q^4*t^5+q^3*t^6+q^2*t^7+q*t^8+t^9+q^8+2*q^7*t+2*q^6*t^2+2*q^5*t^3+2*q^4*t^4+2*q^3*t^5+2*q^2*t^6+2*q*t^7+t^8+q^7+3*q^6*t+4*q^5*t^2+4*q^4*t^3+4*q^3*t^4+4*q^2*t^5+3*q*t^6+t^7+q^6+3*q^5*t+4*q^4*t^2+5*q^3*t^3+4*q^2*t^4+3*q*t^5+t^6+2*q^4*t+3*q^3*t^2+3*q^2*t^3+2*q*t^4+q^3*t+q^2*t^2+q*t^3)*s[2, 1, 1, 1] + (q^8+q^7*t+q^6*t^2+q^5*t^3+q^4*t^4+q^3*t^5+q^2*t^6+q*t^7+t^8+q^7+2*q^6*t+2*q^5*t^2+2*q^4*t^3+2*q^3*t^4+2*q^2*t^5+2*q*t^6+t^7+q^6+3*q^5*t+4*q^4*t^2+4*q^3*t^3+4*q^2*t^4+3*q*t^5+t^6+q^5+3*q^4*t+4*q^3*t^2+4*q^2*t^3+3*q*t^4+t^5+q^4+2*q^3*t+3*q^2*t^2+2*q*t^3+t^4+q^2*t+q*t^2)*s[2, 2, 1] + (q^7+q^6*t+q^5*t^2+q^4*t^3+q^3*t^4+q^2*t^5+q*t^6+t^7+q^6+2*q^5*t+2*q^4*t^2+2*q^3*t^3+2*q^2

In [25]:
_ - s(e[n].nabla())

0

# Debugging stuff

In [ ]:
F.hilbert_polynomial()[Partition([2,2,1])]

In [ ]:
d = (Degrees([0,7]), Partition([2,2,1]))

In [ ]:
F._bases[d].cardinality()

In [ ]:
p1,p2 = F._bases[d]._basis

In [ ]:
p1/=1440;p1

In [ ]:
p2 /= 5040; p2

In [ ]:
R({d[0]:1})

In [ ]:
F.dimensions()[d]

In [ ]:
len(F._todo)

In [ ]:
F.dimensions??

In [ ]:
sum(basis.cardinality() for d, basis in F._bases.iteritems())

In [ ]:
Subspace._extensions

In [ ]:
sum(s(e[n].nabla()).coefficients())(q=1,t=1)

In [ ]:
{d: basis._matrix for d, basis in F._bases.iteritems()}